# The transformative abilities of `sklearn.compose`: a life-saver in disguise?

> Note: the initial working title for this talk was "MLPaaCF: Machine Learning Preprocessing as a Config File," which robbed me of the opportunity to make [Transformers](https://en.wikipedia.org/wiki/Transformers_(film_series)) puns several times throughout.

[Scikit-learn](https://scikit-learn.org/stable/) is undoubtedly one of the most popular libraries for machine learning (ML). From the algorithms provided in its core API to other useful capabilities like feature selection, pipelining, and evaluation, scikit-learn has positioned itself as a must-have on the toolbelt of many data folks. In mid-2018, a new submodule for the core scikit-learn library was initiated: `sklearn.compose`. While still relatively slim, this module, when coupled with existing scikit-learn modules like `sklearn.preprocessing`, can be powerful. The goal of this tutorial is to demonstrate how to implement a configuration-based approach to machine learning dataset creation. Specifically, we'll use the [sklearn.compose](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.compose) and [sklearn.preprocessing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) modules.

The most [recent stable release of scikit-learn](https://scikit-learn.org/dev/versions.html) is version 0.21.3. `sklearn.compose`, by all accounts, seems to have appeared around version 0.20, so the capabilities presented by this section of scikit-learn are relatively new.

## What dataset will we be using?

The [University of California, Irvine (UCI) Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult) contains a treasure-trove of datasets for ML work. I chose the ["Adult" dataset](https://archive.ics.uci.edu/ml/datasets/Adult), which tasks the analyst with predicting, based off of a variety of inputs, whether an adult makes more or less than $50k per year. This dataset comes with a mixture of real, categorical, and integer features, which ought to make for a much more "real-world" dataset-processing example.

## Getting started

If you haven't already, run `sh setup.sh` from the base directory to:

1) Download the "Adult" dataset

2) Set up a virtual environment for dependency management

3) Start the Jupyter Notebook server

In [1]:
def _print(m: str) -> None:
    print(m)

In [2]:
_print('hi')

hi
